## Corona Setup

In [1]:
#Data visualization

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#Audio Analysis
import glob
import IPython
import librosa
import librosa.display
import tensorflow as tf
from tensorflow import keras

import os

In [2]:
#import meta data

df_meta = pd.read_csv('./CoronaHack-Respiratory-Sound-Dataset/Corona-Hack-Respiratory-Sound-Metadata.csv')
#df_meta.info(), df_meta.shape

df_meta['AGE'].value_counts()

25    107
26     78
22     63
30     58
27     57
     ... 
11      1
10      1
8       1
2       1
81      1
Name: AGE, Length: 68, dtype: int64

In [3]:
#Extract positive USER ID
df_meta['split'] = df_meta['COVID_STATUS'].str.split('_').str.get(0)
df_meta_positives = df_meta[df_meta['split'] == 'positive']
df_meta_negatives = df_meta[df_meta['split'] != 'positive']

positives = list(df_meta_positives['USER_ID'])
negatives = list(df_meta_negatives['USER_ID'])
len(positives),len(negatives)
positives

['WNCUJW2QH9fEcsSUztikS1Zvoyn1',
 'TaoyZAahOzRoDQRxb0DtDZh8Opa2',
 'iYM9uJMyO1UoXTo1tc7rXvsI4bm1',
 '1e8i6Q47ewbzrTiKqIeOLEvPv2Z2',
 'G5kUP2QOpVSyjK6Q9rzblmTU9vW2',
 'bH6WQVvClVRDer6iOFvaGiz0luf1',
 '40NRJEGpU9Vg2IFXcuqCQLiKLVz1',
 'wXChdMlWemXwRdX642WnX6JbYfA2',
 'Y635Ri3QFMcOKC0ckXgW2r0xJ422',
 'YHHZYBxPHsUWYITjtMRAixs3DX42',
 'ZzKpUtFGzsVEK7E1wsVGRftKdx93',
 'kBFDtvAVY9QYbi7YHYgd7tNpsWx1',
 'Fx2gbIoPlJXrK5OD8FMeepOGlLa2',
 'HwUhcAKwJIeckUi86cbzb3sTr5m2',
 '9Wkd6QJdpBNEosMpzYhFwZjBJX92',
 'Fi2lLmV3LHR5brvXs0z1JSKDvnI2',
 '05acPS4aRGfvuOfku11Za8zve8i2',
 'VLrOagtGVFYxWiEnMVFn1BnSWHM2',
 'YRiIODBG3RXPbDttwztMhsXg1rv2',
 '6T43bddKoKfG7MwnJWvrPZSsyrc2',
 'hte8VptUoGVFEqvHpbh5brgfcNP2',
 'QjBZv868nydJzk0ZzwgKDHSG6Q82',
 'x8PvE190nSZUtv8raWxfg76ek6p2',
 '3CwioNQVDBQ6CttLyFVRJpMpVHk2',
 'gpcoorcyfuXQZxcUFTxthUKOua03',
 '8fdUOI5UtkZBCVFXNT1a4hl3rX33',
 '6BT19A6gAlNjbtbe3EdgKQc4kkH3',
 'lVHnQ9UNkAYmjNm7wL885LzpsOk2',
 '85iNcV2z6MNT3FrygbEC4fuGrIu1',
 'T190nKNqKoPk02FHlHYJTBDUnPA2',
 'UhYCvhOC

In [80]:
df_meta.head()

,USER_ID,COUNTRY,AGE,COVID_STATUS,ENGLISH_PROFICIENCY,GENDER,COUNTY_RO_STATE,CITY_LOCALITY,Diabetes,Asthma,...,breathing-deep,breathing-shallow,cough-heavy,cough-shallow,counting-fast,counting-normal,vowel-a,vowel-e,vowel-o,split
0,vK2bLRNzllXNeyOMudnNSL5cfpG2,India,24,healthy,Y,M,Karnataka,Bangalore,0,0,...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,healthy
1,bjA2KpSxneNskrLBeqi4bqoTDQl2,India,72,healthy,Y,M,Maharashtra,Thane,0,0,...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,healthy
2,FSzobvJqOXf0rI6X05cHqOiU9Mu2,India,54,healthy,Y,M,Maharashtra,Thane West,0,0,...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,healthy
3,EqDWckxbsETyHUeBLQ8jLtxlhir2,India,31,healthy,Y,M,Karnataka,Bangalore,0,0,...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,healthy
4,FGRDO4IBbAejR0WHD5YbkXTCasg2,India,26,healthy,Y,M,Haryana,gurgaon,0,0,...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,healthy


In [29]:

import tensorflow_io as tfio
import matplotlib.pyplot as plt
from IPython.display import Audio, display

def wav_to_mel(file_path):
  image_target_height, image_target_width = 64, 64
  audio_binary = tf.io.read_file(file_path)
  audio, rate = tf.audio.decode_wav(audio_binary, desired_channels=1)
  #waveform = tf.squeeze(audio, axis=-1)
  audio = audio[:,0]
  print(audio.shape)
  position = tfio.audio.trim(audio, axis=0, epsilon=0.1)
  print(position[0])
  start = position[0]
  end = position[1]
  audio= audio[start:end]

  # Normalize audio data
  audio = tf.cast(audio, tf.float32) / 32768.0  # Max int for audio data
  # Create the spectogram from audio data
  spectrogram = tfio.audio.spectrogram(
      audio, nfft=1024, window=128, stride=64
  )
  # Turn spectrogram into mel spectrogram
  spectrogram = tfio.audio.melscale(
      spectrogram, rate=rate, mels=64, fmin=0, fmax=2000
  )

  #spectrogram /= np.max(audio)
  spectrogram /= tf.math.reduce_max(spectrogram) # Normalize
  spectrogram = tf.expand_dims(spectrogram, axis=-1) # 2D -> 3D
  spectrogram = tf.image.resize(spectrogram, (image_target_height, image_target_width)) # Resize the picture
  spectrogram = tf.transpose(spectrogram, perm=(1, 0, 2)) # Swap the first two axis
  spectrogram = spectrogram[::-1, :, :] # Flip the first axis (frequency)

  display(Audio(audio, rate=8000))
   
  return spectrogram
path = './CoronaHack-Respiratory-Sound-Dataset' + df_meta.iloc[16,'counting-normal']
mel = wav_to_mel(path)
print(mel.shape)
#mel_predict = tf.expand_dims(mel, axis=0)
#print(mel_predict.shape)

#prediction = model.predict(mel_predict)
#print(prediction)
#plt.plot(prediction[0])

plt.imshow(mel[::-1,:], cmap='inferno') #flipping upside down
plt.show()
plt.close()

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [52]:
def preprocess(sample):
    audio_binary = tf.io.read_file(sample)
    audio, rate = tf.audio.decode_wav(audio_binary, desired_channels=1)

    #audio = sample['audio']
    #label = sample['label']
    audio = tf.cast(audio, tf.float32)/ 32768.0 # into float32 and Data types max range for float32 for normalization

    #here it is 1-D sequence of amplitude numbers
    spectrogram = tfio.audio.spectrogram( 
        audio, nfft=1024, window=1024, stride=64
    )

    #here it is 2-D
    spectrogram = tfio.audio.melscale(
        spectrogram, rate=8000, mels=64, fmin=0, fmax=2000 #mels = bins, fmin,fmax = frequences
    )
    #from melscale we need to:
    spectrogram /= tf.math.reduce_max(spectrogram) #normalization
    spectrogram = tf.expand_dims(spectrogram, axis=-1) #add dimension 2D -> 3D
    spectrogram = tf.image.resize(spectrogram, (64,64)) #resize in two dimensions
    
    spectrogram = tf.transpose(spectrogram, perm=(1,0,2)) #transpose the first two axis

    spectrogram = spectrogram[::-1, :, :] #flip the first axis(frequency)
    
    return spectrogram #, label

In [57]:
import cv2
def preprocess_other(sample):
  image_target_height, image_target_width = 64, 64
  audio_binary = tf.io.read_file(sample)
  audio, rate = tf.audio.decode_wav(audio_binary, desired_channels=1)
  #label = sample['label']

  def py_preprocess_audio(audio):
      audio = audio.numpy().astype('float32')

      spectogram = librosa.feature.melspectrogram(
        y=audio, n_fft=1024,  n_mels=64, hop_length=64, sr=8000, fmax=2000 #n_fft = window size, n_mels = frequency bins, hop_lenghth =jump to the right , sr = sound rate, fmax = 
      ) 

      spectogram /= np.max(spectogram) #devide by np.max(audio)
      spectogram = cv2.resize(spectogram, dsize=(image_target_height, image_target_width))
      spectogram = np.expand_dims(spectogram, axis=-1)
      return spectogram

  spectogram = tf.py_function(py_preprocess_audio, [audio], tf.float32)
  spectogram.set_shape((image_target_height, image_target_width, 1))

  return spectogram#, label

In [58]:
path_counnting_normal = df_meta2['counting-normal'].tolist()
path_counnting_normal = ['./CoronaHack-Respiratory-Sound-Dataset' + dir_name for dir_name in path_counnting_normal]
#labels = tf.convert_to_tensor()



# Convert to Tensor
sound_paths_counnting_normal = tf.convert_to_tensor(path_counnting_normal, dtype=tf.string)
# Build a TF Queue, shuffle data
#sound,labels = tf.data.Dataset.from_tensor_slices([sound_paths_counnting_normal, labels])
sound = tf.data.Dataset.from_tensor_slices(sound_paths_counnting_normal)
x = sound.map(lambda sample: preprocess_other(sample)).batch(32)
x

<BatchDataset shapes: (None, 64, 64, 1), types: tf.float32>

In [5]:
#Check for NA
df_meta.loc[:,'counting-normal'].isna().sum()
df_meta.loc[:,'split'].value_counts()

#Generate a dict to categorize split column
cat_dict = {'healthy':'negative','no':'negative','resp':'negative','recovered':'negative','positive':'positive'}

#map cat_dict to split column 
df_meta.loc[:,'split'] =  df_meta.loc[:,'split'].map(cat_dict)
df_meta2 = df_meta.dropna(subset=['split'])



In [95]:
sounds = ['counting-fast','counting-normal']
spects = {}
# initialize with empty list
for i in range(len(df_meta2)):
        spects[i] = []
# create spect for each sound
for sound in sounds:
    for i in range(len(df_meta2)):
        path = './CoronaHack-Respiratory-Sound-Dataset' + df_meta2.loc[i,sound]
        spects[i].append(wav_to_mel(path))

spects   



(196608,)
tf.Tensor(27146, shape=(), dtype=int64)


(405504,)
tf.Tensor(93159, shape=(), dtype=int64)


(417792,)
tf.Tensor(52159, shape=(), dtype=int64)


(421888,)
tf.Tensor(25793, shape=(), dtype=int64)


(176128,)
tf.Tensor(176124, shape=(), dtype=int64)


(237568,)
tf.Tensor(237568, shape=(), dtype=int64)


InvalidArgumentError: Input dimension 1 must have length of at least 1024 but got: 128 [Op:RFFT]

In [ ]:
 !pip install pydub tensorflow_io

In [ ]:
#load data

def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    sound = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f) 
                      for f in os.listdir(label_directory) 
                      if f.endswith(".wav")]
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    return images, labels

ROOT_PATH = "/your/root/path"
train_data_directory = os.path.join(ROOT_PATH, "./CoronaHack-Respiratory-Sound-Dataset/data/train")
test_data_directory = os.path.join(ROOT_PATH, "./CoronaHack-Respiratory-Sound-Dataset/data/test")

images, labels = load_data(train_data_directory)